In [ ]:
read_counts = function(paths){
    
    library(reticulate)
    
    paths = paste('pkl files', paths, sep='')
    pd = import("pandas")
    pickle_data = pd$read_pickle(paths)
    
    return(pickle_data)
    
}

In [ ]:
microarray = function(path){

  df = read.csv(paste('cel files', path, sep=''), sep=',')
  rownames(df) = df$X
  df = df[, -1]

  return(df)
}

In [ ]:
edgeR = function(R, S, name){
    
    library(edgeR)
    
    Res = 'R'
    Sen = 'S'
    
    design = factor(c(rep(Res, ncol(R)), rep(Sen, ncol(S))))
    dataset = cbind(R, S)
                    
    group_data = DGEList(counts=dataset, group=factor(design))                
    normaised_counts = calcNormFactors(group_data)
    common_dispersion_values = estimateCommonDisp(normaised_counts, verbose=F)
    
    gene_exacts_test = exactTest(common_dispersion_values, pair=c(sample_type1,sample_type2))
    
    Differential_genes = topTags(gene_exacts_test, n=Inf, adjust.method='BH', p.value=0.05)
    Differential_genes = as.data.frame(Differential_genes)
    
    up = subset(Differential_genes, logFC > 1)    
    write.csv(up, paste(name, '/DEG.csv', sep=''))
    
    up = R[rownames(up) , ]
    
    return(up)

}

In [ ]:
limma = function(dataset, df1, df2, name){
    
    library(limma)
    library(data.table)
    library(dplyr)
    
    design = model.matrix(~0 + factor(c(rep(1, ncol(df1)), rep(2, ncol(df2)))))
    colnames(design) = c('R', 'S')
    
    fit = lmFit(dataset, design, method='ls')
    contrast.matrix = makeContrasts(Resistant-Sensitive, levels=design)
    
    fit2 = contrasts.fit(fit, contrast.matrix)
    fit2 = eBayes(fit2, trend=TRUE, robust=TRUE)
    
    results = decideTests(fit2, method='separate', adjust.method='BH', p.value=0.05)
    sig = topTable(fit2, n=Inf, adjust='BH', coef=1, sort.by='P', p.value=0.05)
    
    up = subset(sig, logFC > 1)
    write.csv(up, paste(name, '/DEG.csv', sep=''))
    up = df1[rownames(up) , ]
    
    return(up)

}

In [ ]:
power = function(dataset, name){

  library(WGCNA)

  enableWGCNAThreads(nThreads=32)
  allowWGCNAThreads()
  options(stringsAsFactors = FALSE)

  dataset = t(dataset)

  powers = c(c(1:10), seq(from = 12, to=20, by=2))
  sft = pickSoftThreshold(dataset, powerVector = powers, verbose = 5)

  pdf(file=paste('~/Power', name, sep=''))

  par(mfrow = c(1,2))
  plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
       xlab='Soft Threshold (power)', cex.lab=1.2, cex.axis=1.2, cex.main=1.2, cex.sub=1.2,
       ylab='Scale Free Topology Model Fit,signed R^2',type='n',
       main = paste('Scale independence'))

  text(sft$fitIndices[, 1], -sign(sft$fitIndices[, 3])*sft$fitIndices[, 2],
       labels=powers,cex=1, col='red')
  abline(h=0.85,col='red')

  plot(sft$fitIndices[,1], sft$fitIndices[,5], cex.lab=1.2, cex.axis=1.2, cex.main=1.2, cex.sub=1.2,
       xlab='Soft Threshold (power)',ylab='Mean Connectivity', type='n',
       main = paste('Mean connectivity'))

  text(sft$fitIndices[, 1], sft$fitIndices[, 5],
       labels=powers, cex=1, col='red')

  dev.off()

}

In [ ]:
TOM_Matrices = function(dataset, name){

  library(WGCNA)

  enableWGCNAThreads(nThreads=32)
  allowWGCNAThreads()
  options(stringsAsFactors = FALSE)

  dataset = t(dataset)
  adj = abs(cor(dataset, use='p'))^6

  dissTOM = TOMdist(adj)
  hierTOMa = hclust(as.dist(dissTOM), method='average')

  Gene_Modules = labels2colors(cutreeDynamic(hierTOMa, method='tree', cutHeight=0.99))
  Gene_Clusters = labels2colors(cutreeDynamic(hierTOMa, distM= dissTOM , cutHeight = 0.99,
                                               deepSplit=3, pamRespectsDendro = FALSE))


  pdf(file=paste('~/Dendro', name, sep=''))

  plotDendroAndColors(hierTOMa,
                      colors = data.frame(Gene_Clusters),
                      dendroLabels = FALSE,
                      cex.axis = 1.2)

  dev.off()

  return(list(adj, dissTOM, Gene_Modules))

}

In [ ]:
eigenetic_network = function(dataset, colorh1){

  enableWGCNAThreads(nThreads=32)
  allowWGCNAThreads()
  options(stringsAsFactors = FALSE)

  ADJ1 = abs(cor(t(dataset), use='p'))^6

  colors = unique(colorh1)
  Alldegrees1 = intramodularConnectivity(ADJ1, colorh1)

  datME = moduleEigengenes(t(dataset),colorh1)$eigengenes
  MET = orderMEs(cbind(datME))
  datKME = signedKME(t(dataset), datME, outputColumnName='')

  return(datKME)

}

In [ ]:
enrichment = function(dataset, colorh1, datKME){
    
    library(clusterProfiler)
    library(dplyr)
    library(org.Hs.eg.db)
    library(enrichplot)
    
    intModules = table(colorh1)
    intModules = as.data.frame(intModules)
    intModules =intModules$colorh1
    intModules = as.character(intModules)
    
    dat = data.frame()
    dat_new = data.frame()
    
    colrs = c()
    
    for (color in intModules){
        
        color =  color
        FilterGenes = abs(subset(datKME, select=c(color))) > 0.6
        genes = dimnames(data.frame(dataset))[[2]][FilterGenes]
        
        dat = cbind.fill(dat, genes, fill = NA)
        colrs = append(color, colrs)
    
    }
    
    dat = dat[,seq(1,ncol(dat),2)]
    colnames(dat) = colrs
    dat = as.data.frame(dat)
    
    dat = dat[,!names(dat) %in% c("grey")]
    colrs = colnames(dat)
    
    for (j in 1:ncol(dat)){
        
        gene = dat[, j]
        eg = bitr(gene, fromType="ENSEMBL", toType="ENTREZID", OrgDb="org.Hs.eg.db")
        
        genes <- eg$ENTREZID
        dat_new <- cbind.fill(dat_new, genes, fill = NA)
    
    }
    
    dat_new = dat_new[,seq(1, ncol(dat_new), 2)]
    colnames(dat_new) = colrs
    dat_new = as.data.frame(dat_new)
    
    return(dat_new)

}

In [ ]:
clusterPr = function(dat_new){
    
    dat_new = dat_new[,!names(dat_new) %in% c("grey")]
    
    ck = compareCluster(geneCluster = dat_new, fun = "enrichKEGG", pvalueCutoff = 0.05, pAdjustMethod = "BH")
    
    cKEGG = setReadable(ck, OrgDb = org.Hs.eg.db, keyType="ENTREZID")
    cBp = compareCluster(geneCluster = dat_new, fun = "enrichGO", pvalueCutoff = 0.05, OrgDb = org.Hs.eg.db,
                         ont = "BP", pAdjustMethod = "BH")
    
    cGO = setReadable(cBp, OrgDb = org.Hs.eg.db, keyType="ENTREZID")
    
    dotplot(ck)
    dotplot(cGO)

}